We always want our means to be zero and standard deviation to be one.

FastAI v3 8 notes: https://forums.fast.ai/t/lesson-8-notes/41442/22

Normalizing Inputs (C2W1L09): https://www.youtube.com/watch?v=FDCfw-YqWTE

Weight Initialization in a Deep Network (C2W1L11): https://www.youtube.com/watch?v=s2coXdufOzE

****

This is one of my random practice notebook. I try to recreate what I am learning from FastAI without refering the original notes.

The reason to release this publish is to get into the habit of "To be useful: Do everything in public and for the public": https://www.youtube.com/watch?v=5UbixCFJ2DA and to build my deep learning momentum.

Most/all content/ideas from this notebook are not my own but from source are links above. #NoPlagiarism

In [0]:
import torch
from torchvision.datasets import MNIST

In [2]:
data = MNIST('', download=True)

0it [00:00, ?it/s]

9920512it [00:02, 3925970.26it/s]                             


Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 140618.33it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw


1654784it [00:01, 1616285.74it/s]                            
0it [00:00, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


8192it [00:00, 52881.88it/s]            


Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


In [3]:
train = data.train_data
train = train.type(torch.float32)
train = train.view(-1,28*28)
train.shape

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


torch.Size([60000, 784])

In [4]:
perm = torch.randint(low=0, high = 60000, size=(60000,))
perm

tensor([ 7488, 13629,  9165,  ..., 58097, 35750, 27686])

In [0]:
x_train, x_valid = train[perm[:55000]], train[perm[:-5000]]

In [6]:
mean, std = x_train.mean(), x_train.std()
mean, std

(tensor(33.3053), tensor(78.5655))

normalize input

In [0]:
def normalize(data, train_mean, train_std): return (data-train_mean)/train_std

In [8]:
x_train = normalize(x_train, mean, std)
x_train.mean(), x_train.std()

(tensor(0.0001), tensor(1.))

1. Input: 784 Output: 200
2. IP: 200 OP: 50

In [0]:
w1 = torch.randn(784,200)
b1 = torch.zeros(200)
w2 = torch.randn(200, 50)
b2 = torch.zeros(50)

In [0]:
def linear(x, w, b): return x@w + b

In [11]:
x_train.shape, w1.shape

(torch.Size([55000, 784]), torch.Size([784, 200]))

In [12]:
output = linear(x_train, w1, b1)
output.mean(), output.std()

(tensor(-0.3355), tensor(27.5326))

But we want to use relu

In [0]:
def relu(x): return x.clamp_min(0.)

In [14]:
output = relu(output)
output.mean(), output.std()

(tensor(10.6888), tensor(16.0362))

In [15]:
output2 = relu(linear(output, w2, b2))
output2.mean(), output2.std()

(tensor(89.6886), tensor(141.7486))

In [0]:
import math
w1 = torch.randn(784,200)*math.sqrt(2./784) # sqrt(2/input_activation)
w2 = torch.randn(200,50)*math.sqrt(2./200)

In [17]:
output = relu(linear(x_train, w1, b1))
output.mean(), output.std()

(tensor(0.5478), tensor(0.8195))

In [18]:
output2 = relu(linear(output, w2, b2))
output2.mean(), output2.std()

(tensor(0.4655), tensor(0.7520))

Something Jeremy found out was helful:

In [0]:
def relu_better(x): return relu(x) - 0.5

In [20]:
output = relu_better(linear(x_train, w1, b1))
output.mean(), output.std()

(tensor(0.0478), tensor(0.8195))

In [21]:
output2 = relu_better(linear(output, w2, b2))
output2.mean(), output2.std()

(tensor(-0.0596), tensor(0.6680))

Pytorch inbuilt

In [0]:
from torch.nn import init

In [23]:
w1 = torch.randn(784,200)
output = relu(linear(x_train, w1, b1))
output.mean(), output.std()

(tensor(11.7819), tensor(16.9622))

In [24]:
w1 = torch.randn(784,200)
init.kaiming_normal_(w1.T) 
# reason to transpose here is because internally it gets transposed again otherwise m would have been taken as 200
output = relu(linear(x_train, w1, b1))
output.mean(), output.std()

(tensor(0.5422), tensor(0.8150))

Areas of Improvement:

- Spend too much time because I was divinding by sqrt(2/m) instead of multiplying
    - Lesson: Watch formulas carefully
- Learn to use debugger